<a href="https://colab.research.google.com/github/takadeanur/whisper/blob/main/%E3%80%90%E2%98%85%E2%98%85%E5%AE%9F%E8%A1%8C%E5%8F%AF%E8%83%BD%E9%AB%98%E9%80%9F14B%E7%89%88%E3%80%91Wan2_2_14b_light2_APIVer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# 1. 環境準備
# ===============================
!apt -y install -qq aria2

# ===============================
# 2. ComfyUI 本体の取得
# ===============================
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!git reset --hard c60dc4177c16d50da025bda4ac7fd513bb86e699
!pip install -r requirements.txt
!pip install sageattention
!pip install requests --upgrade # FTP/API通信用にrequestsをアップグレード

# ===============================
# 3. カスタムノードの追加
# ===============================
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git

# ---------------- ComfyUI-KJNodesの設定開始 ---------------------
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/kijai/ComfyUI-KJNodes
!pip install -r /content/ComfyUI/custom_nodes/ComfyUI-KJNodes/requirements.txt
%cd /content/ComfyUI/custom_nodes
# ---------------- ComfyUI-KJNodesの設定終了 ---------------------

# ---------------- ComfyUI-Frame-Interpolationの設定開始 ---------------------
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/Fannovel16/ComfyUI-Frame-Interpolation.git
%cd ComfyUI-Frame-Interpolation
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes
# ---------------- ComfyUI-Frame-Interpolationの設定終了 ---------------------


# ===============================
# 4. Diffusion Models (Wan 2.2) ダウンロードとリネーム
# ===============================
# Wan2.2 T2V High Noise (未使用だがダウンロードは維持)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_high_noise_14B_fp8_scaled.safetensors \
    -d /content/ComfyUI/models/diffusion_models -o wan2.2_t2v_high_noise_14B_fp8_scaled.safetensors

# Wan2.2 T2V Low Noise (未使用だがダウンロードは維持)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_low_noise_14B_fp8_scaled.safetensors \
    -d /content/ComfyUI/models/diffusion_models -o wan2.2_t2v_low_noise_14B_fp8_scaled.safetensors

# Wan2.2 I2V High Noise (ファイル名をワークフローに合わせてリネームして保存)
# ワークフロー要求: wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp16.safetensors \
    -d /content/ComfyUI/models/diffusion_models -o wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors

# Wan2.2 I2V Low Noise (ファイル名をワークフローに合わせてリネームして保存)
# ワークフロー要求: wan2.2_i2v_low_noise_14B_fp8_scaled.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_low_noise_14B_fp16.safetensors \
    -d /content/ComfyUI/models/diffusion_models -o wan2.2_i2v_low_noise_14B_fp8_scaled.safetensors

# ===============================
# 5. Text Encoders
# ===============================
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors \
    -d /content/ComfyUI/models/text_encoders -o umt5_xxl_fp8_e4m3fn_scaled.safetensors

# ===============================
# 6. VAE Models
# ===============================
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors \
    -d /content/ComfyUI/models/vae -o wan_2.1_vae.safetensors

# ===============================
# 7. LoRA Models ダウンロードとリネーム
# ===============================
# Lightx2v I2V LoRA (未使用だがダウンロードは維持)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16.safetensors \
    -d /content/ComfyUI/models/loras -o lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16.safetensors

# Wan21 T2V/I2V LoRA (ファイル名をワークフローに合わせてリネームして保存)
# ワークフロー要求: Wan21_T2V_14B_lightx2v_cfg_step_distill_lora_rank32.safetensors
# ワークフローに合わせるため、ダウンロードされるファイルを Wan21_T2V... にリネームします。
# ⚠️ 注意: オリジナルのダウンロードコマンドでは、このファイルはダウンロードされていませんでしたが、ワークフローで使われているため、
# ワークフローで指定されている名前 (Wan21_T2V_14B_lightx2v_cfg_step_distill_lora_rank32.safetensors) で保存するために、
# ファイルパスを元のコードの記述に戻します。
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan21_T2V_14B_lightx2v_cfg_step_distill_lora_rank32.safetensors \
    -d /content/ComfyUI/models/loras -o Wan21_T2V_14B_lightx2v_cfg_step_distill_lora_rank32.safetensors


# ===============================
# 8. 入力素材とワークフローJSONの準備
# ===============================
%cd /content
import os
prefix = "0079"
uuid = "fc496c5f-7ad3-4bdc-89cb-53a2a7d452c5"
root_path = f"https://archive.creativaier.com/comfyui_materials/{prefix}_{uuid}"

# 入力画像ファイルのダウンロード
!rm -rf /content/ComfyUI/input/example.png
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{root_path}/example.png" -d /content/ComfyUI/input -o example.png

# ワークフローJSONファイルのダウンロード (FTP連携では使用しないが、オリジナルコードを維持)
workflow_dir = "/content/ComfyUI/user/default/workflows"
os.makedirs(workflow_dir, exist_ok=True)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{root_path}/text.json" -d "{workflow_dir}" -o text.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{root_path}/image.json" -d "{workflow_dir}" -o image.json

%cd /content/ComfyUI


# ===============================
# 9. FTP連携とAPI実行用のPythonコード
# ===============================
import json
import requests
import time
import subprocess
import threading
import socket
import os
from ftplib import FTP
from google.colab import userdata
import datetime
import random
import shutil

# --- 0. ファイルパスと設定（FTP関連） ---
PORT = 8188
COMFYUI_PORT = PORT
JSON_FILE_NAME = "original_video_workflow.json"
PROMPT_FILE_PATH = f"/content/{JSON_FILE_NAME}"
COMFYUI_SERVER_URL = f"http://127.0.0.1:{COMFYUI_PORT}"

# シークレット情報の取得
print("⚙️ Colab SecretsからFTP接続情報を取得します...")
try:
    FTP_HOST = userdata.get('ftp_host')
    FTP_USER = userdata.get('ftp_user')
    FTP_PASS = userdata.get('ftp_pass')
except:
    FTP_HOST = "your_ftp_host"
    FTP_USER = "your_ftp_user"
    FTP_PASS = "your_ftp_pass"


if not (FTP_HOST and FTP_USER and FTP_PASS) or FTP_HOST == "your_ftp_host":
    print("🚨 警告: FTP接続情報が取得できないか、テスト値のままです。FTP機能は動作しません。")
else:
    print("✅ FTP接続情報の取得に成功しました。")

TODAY_DATE = datetime.date.today().strftime("%Y-%m-%d")
FTP_BASE_DIR = "/jitantech.com/public_html/py-project/youtube-news-project"

INPUT_IMAGE_DIR_NAME = f"target_3d_image/{TODAY_DATE}"
INPUT_IMAGE_FTP_FOLDER = f"{FTP_BASE_DIR}/{INPUT_IMAGE_DIR_NAME}/"

OUTPUT_VIDEO_FTP_FOLDER = f"{FTP_BASE_DIR}/generated_3d_video/"

# ワークフローの LoadImage ノード (ID: 62) が参照するファイル名とパス
COMFYUI_INPUT_FILENAME = "example.png"
INPUT_IMAGE_LOCAL_PATH = f"/content/ComfyUI/input/{COMFYUI_INPUT_FILENAME}"

print(f"⚙️ 処理対象の画像フォルダ: {INPUT_IMAGE_FTP_FOLDER}")
print(f"⚙️ 出力動画の保存先: {OUTPUT_VIDEO_FTP_FOLDER}")

# ワークフローJSONをPythonオブジェクトとして定義
# 【修正済み】: U+00A0エラーを解消し、モデル名も確認済み
workflow = {
    "6": {
        "inputs": {
            "text": "The man walks powerfully towards me.",
            "clip": [
                "38",
                0
            ]
        },
        "class_type": "CLIPTextEncode",
        "_meta": {
            "title": "CLIP Text Encode (Positive Prompt)"
        }
    },
    "7": {
        "inputs": {
            "text": "色调艳丽，過曝，静态，细节模糊不清，字幕，风格，作品，画作，画面，静止，整体发灰，最差质量，低质量，JPEG压缩残留，丑陋的，残缺的，多余の手指，画得不好的手部，画得不好的脸部，畸形的，毁容的，形态畸形の肢体，手指融合，静止不動の画面，杂乱的背景，三条腿，背景人很多，倒着走",
            "clip": [
                "38",
                0
            ]
        },
        "class_type": "CLIPTextEncode",
        "_meta": {
            "title": "CLIP Text Encode (Negative Prompt)"
        }
    },
    "8": {
        "inputs": {
            "samples": [
                "58",
                0
            ],
            "vae": [
                "39",
                0
            ]
        },
        "class_type": "VAEDecode",
        "_meta": {
            "title": "VAEデコード"
        }
    },
    "37": {
        "inputs": {
            "unet_name": "wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors",
            "weight_dtype": "fp8_e4m3fn_fast"
        },
        "class_type": "UNETLoader",
        "_meta": {
            "title": "拡散モデルを読み込む"
        }
    },
    "38": {
        "inputs": {
            "clip_name": "umt5_xxl_fp8_e4m3fn_scaled.safetensors",
            "type": "wan",
            "device": "default"
        },
        "class_type": "CLIPLoader",
        "_meta": {
            "title": "CLIPを読み込む"
        }
    },
    "39": {
        "inputs": {
            "vae_name": "wan_2.1_vae.safetensors"
        },
        "class_type": "VAELoader",
        "_meta": {
            "title": "VAEを読み込む"
        }
    },
    "56": {
        "inputs": {
            "unet_name": "wan2.2_i2v_low_noise_14B_fp8_scaled.safetensors",
            "weight_dtype": "fp8_e4m3fn_fast"
        },
        "class_type": "UNETLoader",
        "_meta": {
            "title": "拡散モデルを読み込む"
        }
    },
    "57": {
        "inputs": {
            "add_noise": "enable",
            "noise_seed": 1042664824122032,
            "steps": 8,
            "cfg": 2,
            "sampler_name": "lcm",
            "scheduler": "simple",
            "start_at_step": 0,
            "end_at_step": 4,
            "return_with_leftover_noise": "enable",
            "model": [
                "71",
                0
            ],
            "positive": [
                "63",
                0
            ],
            "negative": [
                "63",
                1
            ],
            "latent_image": [
                "63",
                2
            ]
        },
        "class_type": "KSamplerAdvanced",
        "_meta": {
            "title": "Kサンプラー（高度）"
        }
    },
    "58": {
        "inputs": {
            "add_noise": "disable",
            "noise_seed": 0,
            "steps": 8,
            "cfg": 2,
            "sampler_name": "lcm",
            "scheduler": "simple",
            "start_at_step": 4,
            "end_at_step": 10000,
            "return_with_leftover_noise": "disable",
            "model": [
                "72",
                0
            ],
            "positive": [
                "63",
                0
            ],
            "negative": [
                "63",
                1
            ],
            "latent_image": [
                "57",
                0
            ]
        },
        "class_type": "KSamplerAdvanced",
        "_meta": {
            "title": "Kサンプラー（高度）"
        }
    },
    "60": {
        "inputs": {
            "fps": 12,
            "images": [
                "8",
                0
            ]
        },
        "class_type": "CreateVideo",
        "_meta": {
            "title": "動画を作成"
        }
    },
    "61": {
        "inputs": {
            "filename_prefix": "ComfyUI_video_output",
            "format": "mp4",
            "codec": "h264",
            "video": [
                "60",
                0
            ]
        },
        "class_type": "SaveVideo",
        "_meta": {
            "title": "Save Video"
        }
    },
    "62": {
        "inputs": {
            "image": COMFYUI_INPUT_FILENAME
        },
        "class_type": "LoadImage",
        "_meta": {
            "title": "画像を読み込む"
        }
    },
    "63": {
        "inputs": {
            "width": 540,
            "height": 960,
            "length": 61,
            "batch_size": 1,
            "positive": [
                "6",
                0
            ],
            "negative": [
                "7",
                0
            ],
            "vae": [
                "39",
                0
            ],
            "start_image": [
                "62",
                0
            ]
        },
        "class_type": "WanImageToVideo",
        "_meta": {
            "title": "Wan画像からビデオへ"
        }
    },
    "67": {
        "inputs": {
            "lora_name": "Wan21_T2V_14B_lightx2v_cfg_step_distill_lora_rank32.safetensors",
            "strength_model": 0.7000000000000002,
            "model": [
                "37",
                0
            ]
        },
        "class_type": "LoraLoaderModelOnly",
        "_meta": {
            "title": "LoRAローダーモデルのみ"
        }
    },
    "68": {
        "inputs": {
            "lora_name": "Wan21_T2V_14B_lightx2v_cfg_step_distill_lora_rank32.safetensors",
            "strength_model": 0.7000000000000002,
            "model": [
                "56",
                0
            ]
        },
        "class_type": "LoraLoaderModelOnly",
        "_meta": {
            "title": "LoRAローダーモデルのみ"
        }
    },
    "69": {
        "inputs": {
            "sage_attention": "auto",
            "model": [
                "67",
                0
            ]
        },
        "class_type": "PathchSageAttentionKJ",
        "_meta": {
            "title": "Patch Sage Attention KJ"
        }
    },
    "70": {
        "inputs": {
            "sage_attention": "auto",
            "model": [
                "68",
                0
            ]
        },
        "class_type": "PathchSageAttentionKJ",
        "_meta": {
            "title": "Patch Sage Attention KJ"
        }
    },
    "71": {
        "inputs": {
            "shift": 8.000000000000002,
            "model": [
                "69",
                0
            ]
        },
        "class_type": "ModelSamplingSD3",
        "_meta": {
            "title": "モデルサンプリングSD3"
        }
    },
    "72": {
        "inputs": {
            "shift": 8,
            "model": [
                "70",
                0
            ]
        },
        "class_type": "ModelSamplingSD3",
        "_meta": {
            "title": "モデルサンプリングSD3"
        }
    }
}

# --- JSON ワークフローを事前に書き出す ---
%cd /content
with open(JSON_FILE_NAME, 'w') as f:
    json.dump(workflow, f, indent=2)
print(f"✅ ワークフローJSONを {PROMPT_FILE_PATH} に書き出しました。")
%cd /content/ComfyUI # ComfyUIディレクトリに戻る


# =======================================================
# 10. FTPからの未処理画像リスト取得
# =======================================================
def get_unprocessed_image_list():
    """
    FTPサーバーから、出力先に同名動画が存在しない未生成の入力画像ファイル（今日付フォルダ内）のリストを取得します。
    """
    print(f"\nFTP サーバー ({INPUT_IMAGE_FTP_FOLDER}) から未生成の入力画像を検索します...")

    try:
        with FTP(FTP_HOST) as ftp:
            ftp.login(user=FTP_USER, passwd=FTP_PASS)
            try:
                files_in_source_full = ftp.nlst(INPUT_IMAGE_FTP_FOLDER)
            except Exception as e:
                raise Exception(f"🚨 FTPソースディレクトリ ({INPUT_IMAGE_FTP_FOLDER}) へのアクセスに失敗しました: 今日付フォルダが存在しないか、アクセスできません。")

            files_in_target_full = []
            try:
                files_in_target_full = ftp.nlst(OUTPUT_VIDEO_FTP_FOLDER)
            except:
                pass

            source_filenames = [os.path.basename(f) for f in files_in_source_full if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            target_filenames_base = [os.path.splitext(os.path.basename(f))[0]
                                     for f in files_in_target_full if f.lower().endswith(('.mp4'))]

            unprocessed_images = []
            for img_file in source_filenames:
                base_name = os.path.splitext(img_file)[0]
                if base_name not in target_filenames_base:
                    unprocessed_images.append(img_file)

            if not unprocessed_images:
                raise Exception("🚨 エラー: 処理すべき新しい画像ファイルがありませんでした。全ての画像は既に動画化されています。")

            print(f"✅ 未処理の画像ファイルを {len(unprocessed_images)} 件見つけました。")
            return unprocessed_images

    except Exception as e:
        raise Exception(f"🚨 FTP (画像リスト取得) 中に致命的なエラーが発生しました: {e}")

# =======================================================
# 11. FTPからの単一画像ダウンロード
# =======================================================
def download_image_to_comfyui_input(ftp_image_name):
    """ 指定されたFTP画像をダウンロードし、ComfyUIの固定入力パスに保存します。"""
    remote_path = os.path.join(INPUT_IMAGE_FTP_FOLDER, ftp_image_name)

    print(f"⚙️ 入力画像 {ftp_image_name} を {INPUT_IMAGE_LOCAL_PATH} にダウンロード中...")
    try:
        with FTP(FTP_HOST) as ftp:
            ftp.login(user=FTP_USER, passwd=FTP_PASS)
            if os.path.exists(INPUT_IMAGE_LOCAL_PATH):
                os.remove(INPUT_IMAGE_LOCAL_PATH)

            with open(INPUT_IMAGE_LOCAL_PATH, 'wb') as f:
                ftp.retrbinary(f'RETR {remote_path}', f.write)

        if not os.path.exists(INPUT_IMAGE_LOCAL_PATH):
            raise FileNotFoundError(f"FTPダウンロード後、ファイル {INPUT_IMAGE_LOCAL_PATH} が見つかりません。")

        print(f"✅ ダウンロード完了。ComfyUIの入力ファイル名: {COMFYUI_INPUT_FILENAME}")
        return True
    except Exception as e:
        print(f"🚨 画像ダウンロード中にエラーが発生しました: {e}")
        return False


# =======================================================
# 12. FTPへの動画アップロード
# =======================================================

def upload_video_to_ftp(local_video_path, ftp_video_name):
    """ 生成された動画をFTPサーバーの指定されたフォルダにアップロードします。"""

    if not local_video_path or not os.path.exists(local_video_path):
        print(f"🚨 エラー: 生成動画ファイル {local_video_path} が見つかりません。アップロードをスキップします。")
        return

    print(f"\nFTP サーバー ({OUTPUT_VIDEO_FTP_FOLDER}) へ生成動画 ({ftp_video_name}) をアップロードします...")

    try:
        with FTP(FTP_HOST) as ftp:
            ftp.login(user=FTP_USER, passwd=FTP_PASS)
            try:
                ftp.mkd(OUTPUT_VIDEO_FTP_FOLDER)
            except Exception:
                pass

            remote_path = os.path.join(OUTPUT_VIDEO_FTP_FOLDER, ftp_video_name)

            with open(local_video_path, 'rb') as file:
                ftp.storbinary(f'STOR {remote_path}', file)

            print(f"✅ 生成動画 {ftp_video_name} を FTP サーバーにアップロードしました。")
    except Exception as e:
        print(f"🚨 FTP (動画アップロード) 中にエラーが発生しました: {e}")


# --- 動画生成 API クライアント関数 ---
def generate_video_with_comfyui(prompt_json_path, server_url, output_video_name):
    """ ローカル ComfyUI API に接続し、動画を生成してローカルファイルパスを返します。"""
    try:
        # 1. ワークフローJSONの読み込み
        with open(prompt_json_path, 'r') as f:
            prompt = json.load(f)

        # 2. KSamplerAdvanced (ノード57, 58) のシードをランダムに設定
        random_seed = random.randint(0, 2**64 - 1)
        prompt['57']['inputs']['noise_seed'] = random_seed
        prompt['58']['inputs']['noise_seed'] = random_seed


        # 3. ワークフロー実行のリクエスト
        client_id = "colab_client_" + str(os.getpid())
        prompt_url = f"{server_url}/prompt"
        payload = {"prompt": prompt, "client_id": client_id}

        print("\n--- ワークフロー送信 ---")

        max_retries = 3
        response = None
        for attempt in range(max_retries):
            try:
                response = requests.post(prompt_url, json=payload, timeout=5)
                response.raise_for_status()
                break
            except requests.exceptions.RequestException as e:
                if attempt < max_retries - 1:
                    print(f"警告: API接続エラー (試行 {attempt + 1}/{max_retries})。再試行します: {e}")
                    time.sleep(2)
                else:
                    raise e

        # 4. 実行完了のポーリング
        prompt_id = response.json().get('prompt_id')
        print(f"実行がキューに追加されました。Prompt ID: {prompt_id}")
        history_url = f"{server_url}/history/{prompt_id}"

        start_poll_time = time.time()
        max_poll_time = 10800 # 3時間

        while time.time() - start_poll_time < max_poll_time:
            time.sleep(2)
            try:
                history_response = requests.get(history_url)
                history_response.raise_for_status()
                if prompt_id in history_response.json():
                    print("動画生成が完了しました。")
                    break
            except requests.exceptions.RequestException:
                continue
        else:
            print(f"🚨 エラー: 動画生成が {max_poll_time} 秒以内に完了しませんでした。")
            return None


        # 5. 動画ダウンロード
        output_info = history_response.json()[prompt_id]['outputs']
        video_data = output_info.get('61', {}).get('videos', [])

        if video_data:
            file_info = video_data[0]
            view_url = f"{server_url}/view"
            params = {
                "filename": file_info['filename'],
                "subfolder": file_info['subfolder'],
                "type": file_info['type'],
                "video": "true"
            }

            print(f"APIから動画ファイルをダウンロード中: {file_info['filename']}")
            video_response = requests.get(view_url, params=params)
            video_response.raise_for_status()

            with open(output_video_name, 'wb') as f:
                f.write(video_response.content)

            print(f"✅ 動画をローカルの {output_video_name} に保存しました。")
            return output_video_name

        # 6. フォールバック: ローカルの最新ファイルを取得
        comfy_output_dir = "/content/ComfyUI/output/"
        latest_video_path = None

        for file in os.listdir(comfy_output_dir):
            if file.endswith(".mp4"):
                file_path = os.path.join(comfy_output_dir, file)
                if not latest_video_path or os.path.getmtime(file_path) > os.path.getmtime(latest_video_path):
                    latest_video_path = file_path

        if latest_video_path:
            print(f"警告: APIダウンロード失敗。ローカル出力ディレクトリから動画ファイルを見つけました: {latest_video_path}")
            shutil.copyfile(latest_video_path, output_video_name)
            return output_video_name

        print("🚨 致命的エラー: 動画ファイル情報が見つかりませんでした。動画が生成されていない可能性があります。")
        return None

    except Exception as e:
        print(f"🚨 処理中に予期せぬエラー: {e}")
        return None


# ===============================
# 13. ComfyUIサーバー起動（cloudflared関連削除済み）
# ===============================
def start_comfyui_server(port):
    print(f"Starting ComfyUI server in API mode on port {port}...")
    # --listen フラグにより、外部からのAPIアクセス（ここではローカルのrequestsによるアクセス）を許可します。
    comfyui_process = subprocess.Popen(
        f"python main.py --listen --port {port}",
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        cwd="/content/ComfyUI"
    )
    return comfyui_process

def wait_for_server(process, port):
    server_url = f"http://127.0.0.1:{port}"
    start_time = time.time()
    max_wait = 60
    print("ComfyUIサーバー起動を待機中...")

    while time.time() - start_time < max_wait:
        try:
            # サーバーのステータスチェック（APIエンドポイントにアクセス）
            response = requests.get(f"{server_url}/queue", timeout=1)
            if response.status_code == 200:
                print("✅ ComfyUIサーバーの起動を確認しました。")
                return True
        except requests.exceptions.RequestException:
            pass

        # サーバーログを読み取り、起動完了のサインを探す
        line = process.stdout.readline()
        if line:
            print(f"ComfyUI Log: {line.strip()}")
            if "To see the GUI go to:" in line or "Unet loaded" in line or "Running on" in line:
                print("サーバーからのログにより、起動確認をしました。")
                return True

        time.sleep(1)
    print("🚨 エラー: ComfyUIサーバーが60秒以内に起動しませんでした。")
    return False

# launch_cloudflared と iframe_thread は完全に削除されています。


# ===============================
# 14. 全体実行フロー
# ===============================
if __name__ == "__main__":

    unprocessed_list = []
    comfyui_process = None

    # 1. FTPから未処理画像のリストを取得
    try:
        unprocessed_list = get_unprocessed_image_list()
    except Exception as e:
        print(f"処理を中断します。エラー: {e}")
        exit()

    # 2. ComfyUIサーバーの起動
    print("\n--- ComfyUIサーバー起動 (API専用) ---")
    comfyui_process = start_comfyui_server(COMFYUI_PORT)

    # 3. サーバー起動確認
    if not wait_for_server(comfyui_process, COMFYUI_PORT):
        if comfyui_process:
            comfyui_process.terminate()
        exit()

    # 4. API クライアントを実行して動画を生成 (リストをループ処理)
    print(f"\n--- 合計 {len(unprocessed_list)} 件の画像に対するループ処理を開始します ---")

    for i, input_image_ftp_name in enumerate(unprocessed_list):
        print(f"\n\n========================================================")
        print(f"== 処理開始: {i + 1}/{len(unprocessed_list)} - {input_image_ftp_name} ==")
        print(f"========================================================")

        base_filename = os.path.splitext(input_image_ftp_name)[0]
        output_video_ftp_name = f"{base_filename}.mp4"
        output_video_name = f"/content/{output_video_ftp_name}"

        # 4a. 入力画像をダウンロード
        if not download_image_to_comfyui_input(input_image_ftp_name):
            continue

        # 4b. API クライアントを実行して動画を生成
        generated_local_path = generate_video_with_comfyui(PROMPT_FILE_PATH, COMFYUI_SERVER_URL, output_video_name)

        # 4c. FTPへ動画をアップロード
        if generated_local_path:
            upload_video_to_ftp(generated_local_path, output_video_ftp_name)

        # 4d. クリーンアップ
        if os.path.exists(INPUT_IMAGE_LOCAL_PATH):
            os.remove(INPUT_IMAGE_LOCAL_PATH)
            print(f"🗑️ {INPUT_IMAGE_LOCAL_PATH} (ComfyUI入力) を削除しました。")
        if os.path.exists(output_video_name):
            os.remove(output_video_name)
            print(f"🗑️️ {output_video_name} (ローカル一時ファイル) を削除しました。")


    # 5. 処理終了後、プロセスを終了
    if comfyui_process:
        comfyui_process.terminate()
        print("\nComfyUI プロセスを終了しました。")

    # 6. 最終クリーンアップ
    print("\n--- 最終クリーンアップ ---")
    if os.path.exists(PROMPT_FILE_PATH):
        os.remove(PROMPT_FILE_PATH)
        print(f"🗑️ {PROMPT_FILE_PATH} を削除しました。")

    print("\n--- 全ての処理が完了しました ---")

try:
    pass
except KeyboardInterrupt:
    print("Execution interrupted by user.")

The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) 